In [1]:
import re
from tools.naver import NaverReportScraper

In [3]:
# 클래스 사용 예시
scraper = NaverReportScraper()
scraper.scrape(start_page=1, end_page=1, include_industry=True)
report_list = scraper.get_reports()
#craper.save_to_db()

산업리포트:   0%|          | 0/49 [00:00<?, ?it/s]Cannot set gray non-stroke color because /'P21' is an invalid float value
Cannot set gray non-stroke color because /'P56' is an invalid float value
Cannot set gray non-stroke color because /'P79' is an invalid float value
Cannot set gray non-stroke color because /'P81' is an invalid float value
Cannot set gray non-stroke color because /'P83' is an invalid float value
Cannot set gray non-stroke color because /'P85' is an invalid float value
Cannot set gray non-stroke color because /'P87' is an invalid float value
Cannot set gray non-stroke color because /'P88' is an invalid float value
Cannot set gray non-stroke color because /'P90' is an invalid float value
Cannot set gray non-stroke color because /'P91' is an invalid float value
Cannot set gray non-stroke color because /'P92' is an invalid float value
Cannot set gray non-stroke color because /'P93' is an invalid float value
Cannot set gray non-stroke color because /'P94' is an invalid float

KeyboardInterrupt: 

In [6]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import io
import time
import random
import os

In [9]:
base_header = {
            "authority": "finance.naver.com",
            "method": "GET",
            "path": "/research/company_list.naver?&page=1",
            "scheme": "https",
            "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
            "accept-encoding": "gzip, deflate, br, zstd",
            "accept-language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
            "cache-control": "no-cache",
            "cookie": "",
            "pragma": "no-cache",
            "priority": "u=0, i",
            "referer": "https://finance.naver.com/research/company_list.naver?&page=2",
            "sec-ch-ua": '"Not)A;Brand";v="8", "Chromium";v="138", "Google Chrome";v="138"',
            "sec-ch-ua-mobile": "?0",
            "sec-ch-ua-platform": "Windows",
            "sec-fetch-dest": "document",
            "sec-fetch-mode": "navigate",
            "sec-fetch-site": "same-origin",
            "sec-fetch-user": "?1",
            "upgrade-insecure-requests": "1",
            "user-agent": os.getenv("USER_AGENT")
        }

In [10]:
session = requests.Session()
industry_url = "https://finance.naver.com/research/industry_list.naver"
req = session.get(industry_url, headers=base_header)
soup = BeautifulSoup(req.text, "html.parser")
table = soup.find("table", class_="type_1")

In [25]:
import warnings
import pdfplumber

def _is_text_page(text):
    """
    페이지 내 텍스트에서 숫자 비율이 50% 이상이면 False 반환 (즉, 차트/표 위주 페이지로 간주)
    """
    if not text or len(text.strip()) == 0:
        return False
    total_chars = len(text)
    num_chars = len(re.findall(r'[0-9]', text))
    if total_chars == 0:
        return False
    return (num_chars / total_chars) < 0.5


def _extract_pdf_text_all_pages(pdf_file):
    """
    pdfplumber PDF 객체에서 모든 페이지의 텍스트를 추출하되,
    각 페이지에서 숫자 비율이 50% 이상인 경우 해당 페이지는 스킵
    """
    text_list = []
    with pdfplumber.open(pdf_file) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text and _is_text_page(page_text):
                text_list.append(page_text)
    return "\n".join(text_list)

In [37]:
rows = table.find_all("tr")
for row in tqdm(rows, desc="산업리포트"):
    cols = row.find_all("td")
    
    if len(cols) < 5:
        continue
    securities = cols[2].get_text(strip=True)  # 증권사
    # 신한투자증권이면 스킵
    if securities == "신한투자증권":
        continue
    
    report_title = cols[1].get_text(strip=True)  # 리포트 이름
    industry_name = cols[0].get_text(strip=True)  # 산업명
    
    # pdf 주소는 .pdf로 끝나는 a 태그에서 추출
    pdf_url = ""
    pdf_a_tag = row.find("a", href=lambda x: x and x.endswith(".pdf"))
    if pdf_a_tag and pdf_a_tag.get("href"):
        pdf_url = pdf_a_tag.get("href")
        if not pdf_url.startswith("http"):
            pdf_url = "https://stock.pstatic.net" + pdf_url
    
    date = cols[4].get_text(strip=True) if len(cols) > 4 else ""

    report_text = ""
    if pdf_url:
        try:
            pdf_req = session.get(pdf_url)
            time.sleep(random.uniform(1, 3))
            pdf_file = io.BytesIO(pdf_req.content)
            # pdfplumber에서 발생하는 UserWarning을 무시
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                report_text = _extract_pdf_text_all_pages(pdf_file)
            if not report_text.strip():
                report_text = "텍스트 없음 또는 PDF 추출 실패"
        except Exception as e:
            report_text = f"PDF 추출 실패: {e}"
    print(report_text)

산업리포트:   0%|          | 0/49 [00:00<?, ?it/s]Cannot set gray non-stroke color because /'P21' is an invalid float value
Cannot set gray non-stroke color because /'P56' is an invalid float value
Cannot set gray non-stroke color because /'P79' is an invalid float value
Cannot set gray non-stroke color because /'P81' is an invalid float value
Cannot set gray non-stroke color because /'P83' is an invalid float value
Cannot set gray non-stroke color because /'P85' is an invalid float value
Cannot set gray non-stroke color because /'P87' is an invalid float value
Cannot set gray non-stroke color because /'P88' is an invalid float value
Cannot set gray non-stroke color because /'P90' is an invalid float value
Cannot set gray non-stroke color because /'P91' is an invalid float value
Cannot set gray non-stroke color because /'P92' is an invalid float value
Cannot set gray non-stroke color because /'P93' is an invalid float value
Cannot set gray non-stroke color because /'P94' is an invalid float

2025년 07월 21일
철강 (Positive)
Steel Alive
1
| C o n t e n t s |
I. 핵심 요약.......................................................................................... 03
II. 투자전략 ........................................................................................... 04
III. 공급단 변화 중심의 업황 회복 국면 진입 ........................................ 08
1. 공급 측면: 중국의 공급 축소 ....................................................... 08
2. 공급 조절의 실현 가능성 및 전망 ............................................... 13
3. 국내 반덤핑 이슈 영향 분석 ........................................................ 19
4. 수요 측면: 중국 부동산 회복, 방향성이 중요 .............................. 23
IV. ‘프리미엄’을 통한 질적성장 .............................................................. 28
1. 국내 철강업계의 수요·공급 구조와 고부가 철강재 ...................... 29
2. 자동차용 고부가 철강재에 주목 .................................................. 32
3. 초고장력강판(AHSS) 수요가 지속될 이유 ................................... 34
4. 현대제철 美 루이지애나 전기로 투자 투자 타당성 분석 ............. 39
V. 기업분석 ..........................

산업리포트:   8%|▊         | 4/49 [00:34<06:10,  8.24s/it]

2025년 07월 18일
ESG
[ESG] 지배구조 규제
대선 이후 규제 동향 Update: 자고 일어나면 새로운 법안이 발의되는 요즘
Analyst 엄수진 sujineom@hanwha.com 3772 7407
1. 상법
7월 3일 통과, 15일 공포된 국회가 3일 본회의를 열고 상법 개정안을 여야 합의로 처리했다. 찬성 220표, 반대 29
상법 개정안 표, 기권 23표로 가결됐다. 금번에 통과된 개정안은 22대 국회에서 발의된 14건의 상법
일부개정법률안의 내용을 통합하여 대안으로 제안된 안이다.
개정의 주요 내용은 이사의 충실의무의 대상을 주주로까지 확대, 감사위원의 사외이사
여부에 관계없이 합산 3% Rule 적용, 전자주주총회의 병행 개최, 사외이사의 명칭 변경
과 비중 상향이다.
정부는 15일에 동 상법 개정안을 국무회의에서 의결했다. 개정 항목별 시행 시기는 차
이가 있는데 주주에 대한 충실의무는 공포 즉시 시행, 정비된 3% Rule과 사외이사 비중
확대 등은 공포 후 1년 뒤, 전자주주총회 병행 개최는 2027년 1월 1일부터이다.
[표1] 상법 개정 Timeline
2024.11.19 이정문 의원 등 18인 상법 일부개정법률안 발의
2024.11.20 법제사법위원회 회부
2024.11.26 제418회 국회(정기회) 제5차 법안심사제1소위
2025.01.22 제421회 국회(임시회) 제1차 법안심사제1소위
2025.02.24 제422회 국회(임시회) 제2차 법안심사제1소위
2025.02.27 국회 본회의 상정 유예
2025.03.13 제423회 국회(임시회) 제1차 본회의 통과
2025.04.01 정부 재의요구권(거부권) 행사
2025.04.17 제424회 국회(임시회) 제5차 본회의 부결
2025.06.03 제21대 대통령선거: 이재명 후보 당선
2025.06.05 이정문 의원 등 25인 상법 일부개정법률안 발의
2025.07.03 상법 일부개정법률안(대안) 제426회 국회(임시회) 제4차 본회의 통과
2025.07.15 국무회의 의결 및 공

산업리포트:  10%|█         | 5/49 [00:39<05:08,  7.00s/it]

...

오리온 2Q25 Re + 삼양식품 급락 코멘트
음식료/담배 Bi-Weekly 냠냠냠 I 2025.07.21
Analyst 심은주 yesej01@hanafn.com RA 고찬결 cgko@hanafn.com
1. 주간 동향 및 이슈
1) 주간 리뷰
 전주 음식료 업종지수는 코스피 대비 1.4%p 언더퍼폼 시현
 전주 상승: 롯데칠성 +6.1%(저가 매수 유입), 노바렉스 +5.1%(수출 호조), 현대그린푸드 +2.8%(저가 매수 유입), 농심
+2.5%(저가 매수 유입), KT&G +2.3%(내년 담뱃세 인상 기대감), CJ프레시웨이 +1.9%(소비쿠폰 수혜 기대감), 대상
+1.3%(저가 매수 유입) 등
 전주 하락: 오리온 -5.0%(2분기 실적 시장 기대 소폭 하회), 삼양식품 -4.6%(단기 급등에 따른 밸류 부담), 빙그레 -
4.6%(차익 실현 매물 출하), 신세계푸드 -2.9%(단기 급등에 따른 밸류 부담), 롯데웰푸드 -2.9%(차익 실현 매물 출하),
동원산업 -2.6%(차익 실현 매물 출하), CJ제일제당 -1.7%(차익 실현 매물 출하) 등
2) 주간 이슈 및 코멘트
<오리온 2Q25 Re>
 오리온의 2분기 연결 매출액 및 영업이익은 각각 7,795억원(YoY 8.1%), 1,245억원(YoY -0.2%)을 시현했다. 손익은 시장
기대를 약 5% 하회했다. 주요 원재료 단가 상승 및 중국 경쟁 강도 강화에 따른 영향으로 해석된다.
 국가별 탑라인은 한국/중국/베트남/러시아가 각각 YoY 4.9%/3.0%/4.3%/64.6%을 기록했다. 로컬 통화기준으로는
중국/베트남/러시아가 각각 YoY 1%/4%/50% 증가한 것으로 추정된다. 한국은 가성비 제품 확대를 통해 매출 성장을 도모
중이다. 중국은 춘절 재고 할인이 일단락 되면서 고성장 채널(ex. 간식 채널)로의 입점 확대에 집중하고 있다. 베트남은 소비
경기가 좋지 않다. 신제품 확대를 통해 수요를 자극 중이다. 러시아는 전년 기저효과(X5 등 일부 채널 단기 납품 중단) 영향도
있

산업리포트:  39%|███▉      | 19/49 [01:50<02:02,  4.10s/it]

21 July, 2025 Yuanta Research
이 안나 2차전지/신에너지
02 3770 5599
anna.lee@yuantakorea.com
이안나
2차전지/신에너지
02 3770 5599
anna.lee@yuantakorea.com
[2차전지 weekly] 투자 전략서란?
이 자료는 2차전지 위클리 data 공유뿐 아니라 주간 섹터 View와 투자아이디어를 공유하는 ‘전략서’입니다.
주간 공유이기 때문에 투자전략은 거의 동일하겠지만 View 가 변하는 시점을 빠르게 공유하고자 함이 자료의
주된 목적입니다.
자료 구성은
1) 투자전략, 2) 주간 핵심 뉴스, 3) 주요 원재료, 소재 가격 data, 4) 셀, 소재 수출 data, 5) 셀, 소재 기업별
Capacity, 6) 상대 주가 추이, 7) Valuation Table로 구성되어 있습니다.
대표 성장 산업인 만큼 많은 이슈, 큰 주가 변동 폭으로 대응이 어려운 섹터입니다. 따라서 ‘[2차전지 weekly]
투자 전략서’를 통해 투자 View를 공유, 이를 통해 투자의 중심을 잡는데 도움이 되고자 합니다.
2차전지 Weekly
투자 전략
섹터 view 하반기 섹터 ‘비중 확대’ 유지! 다만, 3분기 실적 시즌까지 트레이딩 하자
[하반기 2차전지 ‘비중 확대’ 제시하나 산업 회복과 주가 괴리 여전]
* 현실적으로 3분기 실적 발표 시즌까지 트레이딩 예상되는 이유
한국 2차전지 기업의 전략적 재편이 2026년 예상됨
: EV 향 2차전지 시장은 지역별, 세그먼트 별로 뚜렷한 주도권 이동이 가속화. 특히 중국 기업들은
소형 및 중대형 Standard Range EV 시장에서 이미 점유율 확대. 특히, 중국에 이어 유럽까지
생산능력 확장으로 2026년에는 유럽 내 국내 배터리 공장 capa 대비 2배에 이를 것. EU가 중국산
완성품 수입을 제한하더라도, 이미 현지 생산설비가 충분히 구축돼 있고 EV 롱레인지 기술이
경제성을 갖기 전까지는 소형 및 Standard Range EV 중심 수요 증가로 공급 

산업리포트:  39%|███▉      | 19/49 [01:53<02:58,  5.96s/it]


KeyboardInterrupt: 

In [ ]:
pdf_req = session.get(pdf_url)
pdf_file = io.BytesIO(pdf_req.content)

import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", message="Cannot set gray non-stroke color because")
    with pdfplumber.open(pdf_file) as pdf:
        first_page = pdf.pages[0]
        report_text = first_page.extract_text()

In [33]:
import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", message="Cannot set gray non-stroke color because")
    with pdfplumber.open(pdf_file) as pdf:
        first_page = pdf.pages[0]
        report_text = first_page.extract_text()

Cannot set gray non-stroke color because /'P8' is an invalid float value
Cannot set gray non-stroke color because /'P10' is an invalid float value
Cannot set gray non-stroke color because /'P12' is an invalid float value
Cannot set gray non-stroke color because /'P14' is an invalid float value
Cannot set gray non-stroke color because /'P16' is an invalid float value
Cannot set gray non-stroke color because /'P18' is an invalid float value
Cannot set gray non-stroke color because /'P20' is an invalid float value
Cannot set gray non-stroke color because /'P22' is an invalid float value
Cannot set gray non-stroke color because /'P24' is an invalid float value
Cannot set gray non-stroke color because /'P26' is an invalid float value
Cannot set gray non-stroke color because /'P28' is an invalid float value
Cannot set gray non-stroke color because /'P30' is an invalid float value
Cannot set gray non-stroke color because /'P32' is an invalid float value
Cannot set gray non-stroke color becaus

In [36]:
pdf_url

'https://stock.pstatic.net/stock-research/industry/40/20250718_industry_117911000.pdf'

In [35]:
print(report_text)

2025.07.18
IBK투자증권 인터넷/SW/게임
업종지수(전일 종가 기준)
시가총액 Performance(%) PER(배) △EPS PSR ROE
종목 현재가
(조원) -1D -1W -1M -1Y 2024 2025 (%) (배) (%)
한국 코스피 3,192 2,545 0.2 0.3 8.2 12.3 13.6 11.7 24.5 0.8 7.5
코스닥 818 425 0.7 2.6 5.5 -1.3 48.2 28.3 93.7 2.6 0.8
양방향미디어와서비스 2,147 68 -0.8 -5.8 14.8 38.7 29.7 26.2 21.2 3.4 5.7
게임엔터테인먼트 3,165 41 0.5 -1.1 1.1 8.4 23.9 17.7 12.9 3.4 7.8
미국 DOW 44,484 0.5 -0.4 5.4 8.0 22.1 19.4 - 2.8 23.6
NASDAQ 20,886 0.7 1.2 7.0 16.1 32.0 26.3 - 4.9 18.3
Peer Group 및 커버리지 주가
국내 기업동향(전일 종가 기준)
현재가 시가총액 Performance(%) PER(배) △EPS PSR ROE
종목
(원) (십억원) -1D -1W -1M -1Y 2024 2025 (%) (배) (%)
인터넷 NAVER 243,500 38,579 -2.6 -6.2 17.9 40.4 22.2 20.2 1.0 3.7 7.4
SW 카카오 57,500 25,416 2.5 -5.4 11.0 39.6 80.6 52.6 778.8 3.2 3.2
삼성에스디에스 186,200 14,408 -0.6 5.5 17.7 19.3 19.0 17.6 8.5 1.0 8.5
현대오토에버 164,100 4,500 0.2 -2.5 11.5 -3.1 26.7 25.0 5.6 1.3 10.3
더존비즈온 78,800 2,394 -6.9 -2.7 24.1 13.5 44.6 36.8 14.8 6.2 11.9
한글과컴퓨터 28,300 684 -1.9 -5.2 -6.5 39.1 15.5 14.8 232.2 2.2 12.6
롯데이노베이트 22,6

In [ ]:
if table:
                    rows = table.find_all("tr")
                    for row in tqdm(rows, desc="산업리포트"):
                        cols = row.find_all("td")
                        if len(cols) < 5:
                            continue
                        securities = cols[2].get_text(strip=True)  # 증권사

                        # 신한투자증권이면 스킵
                        if securities == "신한투자증권":
                            continue

                        report_title = cols[1].get_text(strip=True)  # 리포트 이름
                        industry_name = cols[0].get_text(strip=True)  # 산업명

                        # pdf 주소는 .pdf로 끝나는 a 태그에서 추출
                        pdf_url = ""
                        pdf_a_tag = row.find("a", href=lambda x: x and x.endswith(".pdf"))

                        if pdf_a_tag and pdf_a_tag.get("href"):
                            pdf_url = pdf_a_tag.get("href")
                            if not pdf_url.startswith("http"):
                                pdf_url = "https://stock.pstatic.net" + pdf_url

                        date = cols[4].get_text(strip=True) if len(cols) > 4 else ""

                        # 오늘 날짜가 아니면 스킵
                        if date != self.today:
                            continue

                        # pdf_url이 있으면 pdf에서 텍스트 추출 (모든 페이지, 숫자비율 50% 이상 페이지는 스킵)
                        report_text = ""
                        if pdf_url:
                            try:
                                pdf_req = session.get(pdf_url)
                                time.sleep(random.uniform(1, 3))
                                pdf_file = io.BytesIO(pdf_req.content)
                                report_text = self._extract_pdf_text_all_pages(pdf_file)
                                if not report_text.strip():
                                    report_text = "텍스트 없음 또는 PDF 추출 실패"
                            except Exception as e:
                                report_text = f"PDF 추출 실패: {e}"